# CODING DOJO 
## Fecha creación:11/3/2022
## Fecha de actualización: 12/3/2022
## Esteban Simbaña

# Limpieza de datos del conjunto de prueba (Starship Exercise)


**En este caso se harán cambios diferentes respecto a la limpieza de datos porque kaggle necesita que no se eliminen filas**

Montamos nuestro drive 

In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importación de librerías

In [45]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np

Importamos la df de prueba

In [46]:
nombrearchivo_df_prueba = '/content/drive/MyDrive/Coding Dojo/Códigos/Track 3/Deberes /Week 2 - Track 3/spaceship-titanic.zip (Unzipped Files)/Df_prueba_convertir/test (1).csv'
df_prueba = pd.read_csv(nombrearchivo_df_prueba)
df_prueba.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


# Limpieza de datos 

##Limpieza de columna home
 
En este caso se analizarán los valores faltantes para considerar eliminar las filas que los contengan, y convertiremos las variables en tipo 'dummies'

In [47]:
#se muestran que valores contiene la columna 'HomePlanet'
df_prueba['HomePlanet'].unique()

array(['Earth', 'Europa', 'Mars', nan], dtype=object)

In [48]:
#Llenamos los valores faltantes con un valor tipo str para poder verificar cuantos valores faltantes hay
df_prueba['HomePlanet'] = df_prueba['HomePlanet'].fillna('Valor_faltante')

In [49]:
#mostramos los diferentes variables dentro de la columna, con sus respectivos contadores de repetición
unique,counts =np.unique(df_prueba['HomePlanet'],return_counts = True)
dict(zip(unique,counts))

{'Earth': 2263, 'Europa': 1002, 'Mars': 925, 'Valor_faltante': 87}

En este caso para no perder información a cada valor faltante los imputaremos con los datos que tenga los mayores contadores en este caso imputaremos los valores faltantes con 'Earth'

In [50]:
for i in range (len(list(df_prueba['HomePlanet']))):
  if df_prueba['HomePlanet'][i] == 'Valor_faltante':
    df_prueba['HomePlanet'][i] = 'Earth'


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [51]:
#Reemplazamos los datos a variables categóricas

df_prueba['HomePlanet'] = df_prueba['HomePlanet'].replace('Earth',1)
df_prueba['HomePlanet'] = df_prueba['HomePlanet'].replace('Mars',0)
df_prueba['HomePlanet'] = df_prueba['HomePlanet'].replace('Europa',2)


In [52]:
df_prueba['HomePlanet'].unique()

array([1, 2, 0])

In [53]:
#finalmente reseteamos los índices para evitar problemas a futuro
df_prueba.reset_index(inplace=True, drop=False)

## Limpuieza de columna Cabin

En este caso, eliminamos la columna completa ya que esta columna presenta datos de la misma naturaleza que 'Passenger_ID'

In [54]:
df_prueba = df_prueba.drop(columns= 'Cabin')

In [55]:
df_prueba.head(1)

,index,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0,0013_01,1,True,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning


## Limpieza de columna 'Age'
En este caso, calcularemos el promedio de la columna 'Age' e imputaremos los valores faltantes con el promedio.

In [56]:
#realizamos una suma acumulada de todos los valores en la columna 'Age'
numerador_prom = df_prueba['Age'].sum()
numerador_prom

119963.0

In [57]:


#para encontrar el denominador, miraremos cuantas filas tiene el df_prueba sin valores faltantes en la columna 'Age'

#asignamos una nueva df_prueba para verificar cuantas filas contiene la columna 'Age' sin valores faltantes

df_prueba_age = df_prueba

#eliminaremos los datos faltantes 

df_prueba_age = df_prueba_age.dropna(subset=['Age'])
#encontraremos cuantas filas tiene el df_prueba_age 

df_prueba_age.shape

(4186, 13)

In [58]:
#automatizamos la obtención del denominador
denominador_prom = list(df_prueba_age.shape)
denominador_prom = denominador_prom[0]
denominador_prom

4186

In [59]:
#obtenemos el valor de la edad promedio
edad_promedio = numerador_prom/denominador_prom
round(edad_promedio)

29

In [60]:
#imputamos los valores en la df_prueba
df_prueba['Age'] = df_prueba['Age'].fillna(edad_promedio)

## Limpieza columna VIP
En este caso imputaremos los valores con False ya que la gran mayoría de datos presenta que son 'False'

In [61]:
#cambiaremos los valores a tipo string ya que hay valores booleanos 
df_prueba['VIP']= df_prueba['VIP'].astype(str)

In [62]:
#verificamos que la mayoria de los valores son falsos 
a,b =np.unique(df_prueba['VIP'],return_counts = True)
dict(zip(a,b))

{'False': 4110, 'True': 74, 'nan': 93}

Como podemos observar es una buena estrategia imputar las filas con datos faltantes con el valor de 'False' ya que constituyen más del 90% 

In [65]:
#cambiamos los valores de 'nan' a 'False'
for i in range (len(list(df_prueba['VIP']))):
  if df_prueba['VIP'][i] == 'nan' :
    df_prueba['VIP'][i] = 'False'


In [66]:
#convertimos en variables categóricas las variables en la columna 'VIP'

#Reemplazar los valores a 0s y 1s
df_prueba['VIP'] = df_prueba['VIP'].replace('True',1)
df_prueba['VIP'] = df_prueba['VIP'].replace('False',0)

In [67]:
#Verficamos los cambios 

df_prueba.head(3)

,index,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0,0013_01,1,True,TRAPPIST-1e,27.0,0,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,1,0018_01,1,False,TRAPPIST-1e,19.0,0,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,2,0019_01,2,True,55 Cancri e,31.0,0,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus


## Limpieza columnas 'RoomService' , 'FoodCourt' , 'ShppingMall' y otras.

En este caso miraremos que columnas contienen una correlación positiva con nuestro vector objetivo "Transported"




Como no tienen una alta correlación negativa se mantienen las variables e imputaremos los valores con 0 ya que es el dato que más se repite en cada una de las columnas de gastos

In [68]:
df_prueba.isnull().sum()

index             0
PassengerId       0
HomePlanet        0
CryoSleep        93
Destination      92
Age               0
VIP               0
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

In [69]:
#Rellanamos los datos faltantes con 0 en las columnas que se traten de gastos 
df_prueba['RoomService'] = df_prueba['RoomService'].fillna(0)
df_prueba['FoodCourt'] = df_prueba['FoodCourt'].fillna(0)
df_prueba['ShoppingMall'] = df_prueba['ShoppingMall'].fillna(0)
df_prueba['Spa'] = df_prueba['Spa'].fillna(0)
df_prueba['VRDeck'] = df_prueba['VRDeck'].fillna(0)

In [70]:
df_prueba.isnull().sum()

index            0
PassengerId      0
HomePlanet       0
CryoSleep       93
Destination     92
Age              0
VIP              0
RoomService      0
FoodCourt        0
ShoppingMall     0
Spa              0
VRDeck           0
Name            94
dtype: int64

##Limpieza columna destination

En este caso aplicaremos un proceso similar al de la columna 'HomePlanet' que se realizó previamente

In [71]:
#Llenamos los valores faltantes con un valor tipo str para poder verificar cuantos valores faltantes hay
df_prueba['Destination'] = df_prueba['Destination'].fillna('Valor_faltante')

In [72]:
#Verificamos que y cuantos tipos de valores hay en la columna 'Destination'
a,b =np.unique(df_prueba['Destination'],return_counts = True)
dict(zip(a,b))

{'55 Cancri e': 841,
 'PSO J318.5-22': 388,
 'TRAPPIST-1e': 2956,
 'Valor_faltante': 92}

In [73]:
#cambiamos los datos con valor 'Valor_Faltante' a 'TRAPPIST-1e'
for i in range (len(list(df_prueba['Destination']))):
  if df_prueba['Destination'][i] == 'Valor_faltante':
    df_prueba['Destination'][i] = 'TRAPPIST-1e'


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [74]:
#Reemplazamos los datos a variables categóricas

df_prueba['Destination'] = df_prueba['Destination'].replace('55 Cancri e',1)
df_prueba['Destination'] = df_prueba['Destination'].replace('PSO J318.5-22',0)
df_prueba['Destination'] = df_prueba['Destination'].replace('TRAPPIST-1e',2)


In [75]:
#verificamos los cambios 
df_prueba['Destination'].unique()

array([2, 1, 0])

In [76]:
#reseteamos los índices
df_prueba.reset_index(inplace=True, drop=False)

## Imputar los valores faltantes y convertir a variables dummies los datos en la columna 'CryOsleep'

In [77]:
df_prueba.head(1)

,level_0,index,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0,0,0013_01,1,True,2,27.0,0,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning


In [78]:
#convertimos las variables a tipo string para poder verificar cuantos valores contiene 
df_prueba['CryoSleep'] = df_prueba['CryoSleep'].astype(str)

In [79]:
#Llenamos los valores faltantes con un valor tipo str para poder verificar cuantos valores faltantes hay
df_prueba['CryoSleep'] = df_prueba['CryoSleep'].fillna('Valor_faltante')

In [80]:
#Verificamos que y cuantos tipos de valores hay en la columna 'Destination'
a,b =np.unique(df_prueba['CryoSleep'],return_counts = True)
dict(zip(a,b))

{'False': 2640, 'True': 1544, 'nan': 93}

En este caso por tener más datos 'False' se impputarán el valor de 'False' en las filas.

In [82]:
#cambiamos los datos con valor 'Valor_Faltante' a 'False'
for i in range (len(list(df_prueba['CryoSleep']))):
  if df_prueba['CryoSleep'][i] == 'nan':
    df_prueba['CryoSleep'][i] = 'False'

In [83]:
#Reemplazar los valores a 0s y 1s en la columna 'CryoSleep'
df_prueba['CryoSleep'] = df_prueba['CryoSleep'].replace('True',1)
df_prueba['CryoSleep'] = df_prueba['CryoSleep'].replace('False',0)

In [84]:
df_prueba.head()

,level_0,index,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0,0,0013_01,1,1,2,27.0,0,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,1,1,0018_01,1,0,2,19.0,0,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,2,2,0019_01,2,1,1,31.0,0,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,3,3,0021_01,2,0,2,38.0,0,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,4,4,0023_01,1,0,2,20.0,0,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [85]:
df_prueba.shape

(4277, 14)

Finalmente exportamos la df para poder usarla como df de prueba 

In [86]:
df_prueba.isnull().sum()

level_0          0
index            0
PassengerId      0
HomePlanet       0
CryoSleep        0
Destination      0
Age              0
VIP              0
RoomService      0
FoodCourt        0
ShoppingMall     0
Spa              0
VRDeck           0
Name            94
dtype: int64

In [87]:
df_prueba.to_csv('/content/drive/MyDrive/Coding Dojo/Códigos/Track 3/Deberes /Week 2 - Track 3/spaceship-titanic.zip (Unzipped Files)/Df_prueba_convertir/df_prueba.csv')